#**1. 결정트리**
- sklearn 패키지의 breast_cancer 데이터셋 이용

##**1) 데이터를 로딩하고, 훈련세트와 테스트 세트로 나누기(훈련 세트 : 80%)**

In [3]:
## 데이터 준비
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# 데이터 로드
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# 데이터를 훈련 세트와 테스트 세트로 나누기 (80:20 비율,random_state=42)
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42)

##**2) 결정 트리 모델을 생성 및 학습하기**

In [5]:
from sklearn.tree import DecisionTreeClassifier

# 결정 트리 모델 생성
dt_clf = DecisionTreeClassifier(random_state=42)

# 결정 트리 답변1&2(모델생성, 모델학습)
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

##**3) 테스트 데이터로 예측 정확도를 구하기**

In [7]:
from sklearn.metrics import accuracy_score
# 테스트 세트에 대한 예측 수행 > predict() 사용
pred = dt_clf.predict(X_test)
# accuracy_score 속성 사용
accuracy= accuracy_score(y_test, pred)
print('결정 트리 예측 정확도 : {0:.4f}'.format(accuracy))

결정 트리 예측 정확도 : 0.9474


##**4) 오차행렬 출력하기**

In [9]:
from sklearn.metrics import confusion_matrix

# 오차 행렬 출력
conma = confusion_matrix(y_test, pred)
print("오차 행렬:\n", conma)

오차 행렬:
 [[40  3]
 [ 3 68]]


##**5) 하이퍼 파라미터 튜닝**
### 파라미터 조합
    params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
    }


In [11]:
from sklearn.model_selection import GridSearchCV
params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
    }
# GridSearchCV 객체 생성 (교차 검증은 cv=5,verbose=1)
grid_cv = GridSearchCV(dt_clf, param_grid=params, cv=5, verbose = 1, scoring='accuracy')

# 훈련 데이터를 사용하여 GridSearchCV 모델 학습 > fit() 사용
grid_cv.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 7, 10, None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy', verbose=1)

##**6) 최적의 하이퍼 파라미터 점수와 평균 정확도 수치 출력**
 예시 :
- GridSearchCV 최적 하이퍼 파라미터 :
  {'criterion': ,'max_depth': ,'min_samples_leaf': ,'min_samples_split':}
- 최고의 교차 검증 점수: 0.xxx

In [13]:
print("최적의 하이퍼파라미터:", grid_cv.best_params_)
print("최고의 교차 검증 점수:{0:.4f}".format(grid_cv.best_score_))

최적의 하이퍼파라미터: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10}
최고의 교차 검증 점수:0.9473


##**7) 최적화된 모델로 테스트 데이터 예측 수행**
- 최적 하이퍼 파라미터로 학습은 어떻게? : best_estimator_ 속성 활용

In [15]:
# 최적의 하이퍼 파라미터 학습 코드
best_dt_clf = grid_cv.best_estimator_
# 테스트에 대한 예측 수행 코드
pred1 = best_dt_clf.predict(X_test)
# accuracy_score 속성 사용
accuracy = accuracy_score(y_test, pred1)
print('결정 트리 예측 정확도 : {0:.4f}'.format(accuracy))

결정 트리 예측 정확도 : 0.9561


##**8) 특성 중요도 시각화**
- top 6 피처를 막대그래프로 표현하기
- 각 피처의 중요도 : feature_importances_ 속성으로
- numpy로 피처 이름과 중요도 정렬하기

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# 최적의 모델에서 피처 중요도 추출
feature_importances_values = best_df_clf.feature_importances_
# 유방암 데이터셋에서 피처 네임 추출 >> feature_names 속성이용
features = np.array(cancer.feature_names)

# 중요도 순으로 내림차순 정렬 >> 넘파이의 argsort()
ranking = np.argsort(feature_importances_values)[::-1]

# 상위 6개의 피처 선택 >> 앞의 내림차순된 ranking 배열에서 인덱스 추출
top6_ranking = ranking[:6]

# 상위 6개 피처와 중요도 값을 추출
top6_features = feature[top6_ranking]
top6_importances = feature_importances_values[top6_ranking]

# 상위 6개 피처 중요도 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=top6_importances, y=top6_features)
plt.title('Feature Importance Analysis Top 6')
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.show()

#**2. 앙상블**

In [20]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

##**1) digits 데이터 셋을 load_digits() 함수를 사용해서 불러온 후, 훈련 세트와 테스트 세트로 나눠 주세요.**

- test_size = 0.2
- random_state = 42

In [22]:
# 데이터셋 로드 및 훈련/테스트 세트 분리
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)

##**2)로지스틱 회귀, KNN, 결정 트리를 개별 모델로 사용하여 개별 모델의 학습/예측/평가를 진행해 주세요.**

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

- 로지스틱 회귀: max_iter = 10000, solver = 'lbfgs'
- KNN: n_neighbors = 16
- 결정 트리: random_state = 12

In [26]:
# 개별 모델 선언
lr_clf = LogisticRegression(max_iter=10000, solver='lbfgs')
knn_clf = KNeighborsClassifier(n_neighbors=16)
dt_clf = DecisionTreeClassifier(random_state=12)

# 각 모델에 대해 학습 및 평가 수행
classifiers =[lr_clf, knn_clf, dt_clf]
for classifier in classifiers :
  classifier.fit(X_train, y_train)
  pred = classifier.predict(X_test)
  class_name = classifier.__class__.__name__
  print("{0} 정확도: {1:.4f}".format(class_name, accuracy_score(y_test, pred)))

LogisticRegression 정확도: 0.9750
KNeighborsClassifier 정확도: 0.9806
DecisionTreeClassifier 정확도: 0.8639


##**3)앞서 만든 모델 3개를 기반으로 하여 '하드 보팅' 방식으로 보팅 분류기를 생성한 후, 학습/예측/평가를 진행해 주세요.**

In [28]:
from sklearn.ensemble import VotingClassifier

In [30]:
# 하드 보팅 분류기 생성
hard_clf = VotingClassifier(estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)], voting='hard')
hard_clf.fit(X_train, y_train)
hard_pred = hard_clf.predict(X_test)
print("Hard Voting Accuracy:", accuracy_score(y_test, hard_pred))

Hard Voting Accuracy: 0.975


##**4)앞서 만든 모델 3개를 기반으로 하여 '소프트 보팅' 방식으로 보팅 분류기를 생성한 후, 학습/예측/평가를 진행해 주세요.**

In [32]:
# 소프트 보팅 분류기 생성
soft_clf = VotingClassifier(estimators=[('lr', lr_clf), ('knn', knn_clf), ('dt', dt_clf)], voting='soft') 
soft_clf.fit(X_train, y_train)
soft_pred = soft_clf.predict(X_test)
print("Soft Voting Accuracy:", accuracy_score(y_test, soft_pred))

Soft Voting Accuracy: 0.9777777777777777


#**3. OOB 평가**

- n_estimators = 100
- 중복 허용 가능
- 가용한 모든 코어 사용
- random_state=42

In [34]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Bagging을 사용한 결정 트리 학습 및 OOB 평가
bag_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    bootstrap=True,  
    oob_score=True, 
    n_jobs=-1, 
    random_state=42
)

bag_clf.fit(X_train, y_train)

# OOB 평가 점수 출력
print("OOB Score:", bag_clf.oob_score_)


### **4-4 랜덤포레스트**
사이킷런의 붓꽃데이터세트를 RandomForestClassifier를 이용하여 예측하세요##**1) 사이킷런의 붓꽃데이터세트를 RandomForestClassifier를 이용하여 예측하세요.**

In [38]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# 데이터셋 로드
iris = load_iris()
X = iris.data
y = iris.target

# 데이터셋을 훈련 세트와 테스트 세트로 분리 (test_size =0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 생성 및 훈련(n_estimators=100, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 테스트 세트로 예측
y_pred = model.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f'예측 정확도: {accuracy:.2f}')

예측 정확도: 1.00


사이킷런의 유방암 데이터셋을 이용하여 GridSearchCV기법으로 최적파라미터를 구하는 코드를 짜세요

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 데이터셋 로드
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# 데이터셋을 훈련 세트와 테스트 세트로 분리 (test_size =0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 생성(random_stae=42)
model = RandomForestClassifier(random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# GridSearchCV를 사용하여 최적 파라미터 찾기(cv=5, n_jobs=-1, verbose=2)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 및 최고 예측정확도
print('최적 하이퍼파라미터:', ##답변##)
print('최고 예측 정확도: {:.4f}'.format(##답변##))

# 최적 파라미터로 모델 훈련 및 테스트
best_model = ##답변##
y_pred = ##답변##

# 테스트 세트 정확도 평가
accuracy = ##답변##
print(f'테스트 세트 예측 정확도: {accuracy:.2f}')